In [ ]:
from fetch_data import fetch_data_from_postgres
#Merge data
mvariables = {
    "ecosystem" = "Urban-City"
    "lat_min": 48.815573,
    "lat_max": 48.902145,
    "lon_min": 2.224199,
    "lon_max": 2.469921,
    "years": [2020]
}

json_outputs = fetch_data_from_postgres(variables=mvariables)


--- DB SIGN IN: ---
reading .env files...
successfully connected to:	 postgres
available schemas:		 ecosystem_classifier, information_schema, public
setting default schema to:	 ecosystem_classifier
tables in default schema:	 radiance_2020, ndvi_2020, earthdata_1980, meteostat_2017, us_rules, meteostat_2010, meteostat_2012, meteostat_2013, meteostat_2014, meteostat_2018, meteostat_2000, elevation, us_gaz, earthdata_1990, earthdata_1995, us_lex, meteostat_2015, earthdata_2011, earthdata_2012, meteostat_1990, meteostat_2011, meteostat_stations, earthdata_1970, earthdata_2019, radiance_2017, radiance_2018, radiance_2019, radiance_2021, earthdata_2000, earthdata_2005, earthdata_2010, earthdata_2013, meteostat_2016, spatial_ref_sys, app_contact_messages, earthdata_2014, meteostat_2019, meteostat_2020, meteostat_2021, earthdata_2015, earthdata_2016, earthdata_2017, earthdata_2018, meteostat_2005, radiance_2012, radiance_2013, radiance_2014, radiance_2015, radiance_2016, radiance_2022, radian

In [2]:
import json
import pandas as pd

# Initialize an empty list to store the DataFrames
dfs = []

# Loop over each table in json_outputs
for key, data in json_outputs.items():
    # Parse JSON string if necessary
    if isinstance(data, str):
        try:
            parsed_data = json.loads(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for table '{key}': {e}")
            continue  # Skip this table if JSON parsing fails
    else:
        parsed_data = data

    # Convert parsed data to a DataFrame
    try:
        df = pd.DataFrame(parsed_data)
    except ValueError as e:
        print(f"Error creating DataFrame for table '{key}': {e}")
        continue  # Skip this table if DataFrame creation fails

    # Standardize column names to 'lat' and 'lon' for merging
    if 'latitude' in df.columns and 'longitude' in df.columns:
        df.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)
    elif 'lat' not in df.columns or 'lon' not in df.columns:
        print(f"Warning: Table '{key}' is missing required 'lat'/'lon' or 'latitude'/'longitude' columns.")
        continue  # Skip tables without valid latitude/longitude columns

    # Append the DataFrame to the list
    dfs.append(df)

# Merge all DataFrames on 'lat' and 'lon' if there are valid DataFrames
if dfs:
    merged_df = dfs[0]
    for df in dfs[1:]:
        try:
            merged_df = merged_df.merge(df, on=['lat', 'lon'], how='outer')
        except KeyError as e:
            print(f"Error merging table due to missing 'lat'/'lon' columns: {e}")
            continue  # Skip merging if 'lat'/'lon' columns are missing

    # Display the merged DataFrame
    print("\nMerged DataFrame:")
    print(merged_df.head(10))
else:
    print("No valid tables with latitude and longitude data were found.")



Merged DataFrame:
      lat    lon  elevation   radiance      ndvi  Swnet_min  Swnet_max  \
0  48.875  2.375       78.0  50.000786  0.394561  23.804476  204.75125   
1  48.875  2.375       78.0  50.000788  0.394561  23.804476  204.75125   

   Swnet_avg  Lwnet_min  Lwnet_max  ...  Qair_avg  Psurf_min   Psurf_max  \
0  114.74509 -82.530136   -34.4757  ...  0.006318   99721.39  101450.805   
1  114.74509 -82.530136   -34.4757  ...  0.006318   99721.39  101450.805   

   Psurf_avg  SWdown_min  SWdown_max  SWdown_avg  LWdown_min  LWdown_max  \
0  100724.99   26.911814   242.07588   133.78767   294.61588   369.89417   
1  100724.99   26.911814   242.07588   133.78767   294.61588   369.89417   

   LWdown_avg  
0   325.08878  
1   325.08878  

[2 rows x 113 columns]


In [3]:
merged_df["ecosystem"] = "Urban-City"